## Implementing binary decision trees with real valued features
**Multi class supported - Classes IDs are 0 through C**

In [3]:
from sklearn.datasets import make_classification
from sklearn.cross_validation import train_test_split
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import division
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from tensorflow.examples.tutorials.mnist import input_data

### generating data

In [172]:
mnist = input_data.read_data_sets("../MNIST_data/", one_hot=False)
Xtrain = mnist.train.images
Xtest = mnist.test.images
ytrain = mnist.train.labels
ytest = mnist.test.labels
Xsource, Xtarget, ysource, ytarget = train_test_split(Xtest, ytest, test_size=0.3)
print Xtrain.shape
print Xtest.shape
print Xsource.shape

Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz
(55000, 784)
(10000, 784)
(7000, 784)


### Writing in line for decision tree

In [153]:
from sklearn.tree import DecisionTreeClassifier
estimator = DecisionTreeClassifier(max_leaf_nodes=6, random_state=0)
estimator.fit(Xsource, ysource)

# The decision estimator has an attribute called tree_  which stores the entire
# tree structure and allows access to low level attributes. The binary tree
# tree_ is represented as a number of parallel arrays. The i-th element of each
# array holds information about the node `i`. Node 0 is the tree's root. NOTE:
# Some of the arrays only apply to either leaves or split nodes, resp. In this
# case the values of nodes of the other type are arbitrary!
#
# Among those arrays, we have:
#   - left_child, id of the left child of the node
#   - right_child, id of the right child of the node
#   - feature, feature used for splitting the node
#   - threshold, threshold value at the node
#

# Using those arrays, we can parse the tree structure:

n_nodes = estimator.tree_.node_count
children_left = estimator.tree_.children_left
children_right = estimator.tree_.children_right
feature = estimator.tree_.feature
threshold = estimator.tree_.threshold


# The tree structure can be traversed to compute various properties such
# as the depth of each node and whether or not it is a leaf.
node_depth = np.zeros(shape=n_nodes)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, -1)]  # seed is the root node id and its parent depth
while len(stack) > 0:
    node_id, parent_depth = stack.pop()
    node_depth[node_id] = parent_depth + 1

    # If we have a test node
    if (children_left[node_id] != children_right[node_id]):
        stack.append((children_left[node_id], parent_depth + 1))
        stack.append((children_right[node_id], parent_depth + 1))
    else:
        is_leaves[node_id] = True

print("The binary tree structure has %s nodes and has "
      "the following tree structure:"
      % n_nodes)
for i in range(n_nodes):
    if is_leaves[i]:
        print("%snode=%s leaf node." % (node_depth[i] * "\t", i))
    else:
        print("%snode=%s test node: go to node %s if X[:, %s] <= %ss else to "
              "node %s."
              % (node_depth[i] * "\t",
                 i,
                 children_left[i],
                 feature[i],
                 threshold[i],
                 children_right[i],
                 ))
print()

#rint("It is %s %% of all nodes." % (100 * len(common_node_id) / n_nodes,))

The binary tree structure has 11 nodes and has the following tree structure:
node=0 test node: go to node 1 if X[:, 350] <= 0.331372559071s else to node 2.
	node=1 test node: go to node 3 if X[:, 436] <= 0.00196078442968s else to node 4.
	node=2 test node: go to node 5 if X[:, 489] <= 0.00588235352188s else to node 6.
		node=3 leaf node.
		node=4 test node: go to node 7 if X[:, 542] <= 0.00588235352188s else to node 8.
		node=5 leaf node.
		node=6 leaf node.
			node=7 test node: go to node 9 if X[:, 432] <= 0.00588235352188s else to node 10.
			node=8 leaf node.
				node=9 leaf node.
				node=10 leaf node.
()


/Users/RoozbehFarhoudi/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:53: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/RoozbehFarhoudi/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:49: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


### code for converting to dictionary format

In [173]:
def convert_from_scikit_learn_to_dic_ite(node_index,is_leaves, children_left,children_right,feature,threshold):
        
        a = is_leaves[0]
        b = feature[0]
        c = threshold[0]
        if (a):
            return {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True,
            'prediction': None,
            'labels_distribution':None}
    
        else:
            left = children_left[0]-node_index[0]
            if(left==-1):
                left_tree = None
            else:
                left_tree = convert_from_scikit_learn_to_dic_ite(node_index[left:],is_leaves[left:], children_left[left:],children_right[left:],feature[left:],threshold[left:])
            right = children_right[0]-node_index[0]
            if(right==-1):
                right_tree = None
            else:
                right_tree = convert_from_scikit_learn_to_dic_ite(node_index[right:],is_leaves[right:], children_left[right:],children_right[right:],feature[right:],threshold[right:])
            return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': b,
            'threshold'        : c,
            'left'             : left_tree, 
            'right'            : right_tree,
            'labels_distribution': None}


def convert_from_scikit_learn_to_dic(tree):

    n_nodes = tree.tree_.node_count
    children_left = tree.tree_.children_left
    children_right = tree.tree_.children_right
    feature = tree.tree_.feature
    threshold = tree.tree_.threshold
    node_index = np.array(range(0,n_nodes))

# The tree structure can be traversed to compute various properties such
# as the depth of each node and whether or not it is a leaf.
    node_depth = np.zeros(shape=n_nodes)
    is_leaves = np.zeros(shape=n_nodes, dtype=bool)
    stack = [(0, -1)]  # seed is the root node id and its parent depth
    while len(stack) > 0:
        node_id, parent_depth = stack.pop()
        node_depth[node_id] = parent_depth + 1

    # If we have a test node
        if (children_left[node_id] != children_right[node_id]):
            stack.append((children_left[node_id], parent_depth + 1))
            stack.append((children_right[node_id], parent_depth + 1))
        else:
            is_leaves[node_id] = True
    
    return convert_from_scikit_learn_to_dic_ite(node_index,is_leaves, children_left,children_right,feature,threshold)

In [174]:
a = convert_from_scikit_learn_to_dic(estimator)
estimator = DecisionTreeClassifier(max_leaf_nodes=6, random_state=0)
estimator.fit(Xsource, ysource)
a['left']['right']['left']

{'is_leaf': False,
 'labels_distribution': None,
 'left': {'is_leaf': True,
  'labels_distribution': None,
  'left': None,
  'prediction': None,
  'right': None,
  'splitting_feature': None},
 'prediction': None,
 'right': {'is_leaf': True,
  'labels_distribution': None,
  'left': None,
  'prediction': None,
  'right': None,
  'splitting_feature': None},
 'splitting_feature': 432,
 'threshold': 0.0058823535218834877}

### Decision Tree functions

In [100]:
def intermediate_node_num_mistakes(labels_in_node):
    
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0
    
    C,unique_counts = np.unique(labels_in_node,return_counts=True) #the id of classes and number of each
    
    return (len(labels_in_node) - unique_counts[np.argmax(unique_counts)])


def reached_minimum_node_size(y, min_node_size):
    # Return True if the number of data points is less than or equal to the minimum node size.
    if y.shape[0] <= min_node_size:
        #print y.shape[0]
        return True

    
# X matrix of features (p datapoints x N features)
# y vector of labels (p x 1)

def best_splitting_feature(X, y, Nbins):
        
    best_feature = None # Keep track of the best feature 
    best_threshold = None
    best_I = -1     # Keep track of the best info gain so far 

    #the number of data points in the parent node
    num_data_points = y.shape[0]
    
    # Loop through each feature to consider splitting on that feature
    for feature in range(X.shape[1]):
        
        fvals = X[:,feature]
        fvals = np.sort(fvals)  #sorting the values
        if num_data_points > Nbins:            
            fvals = fvals[range(0,num_data_points,Nbins)]
        
        #loop through all values of current feature to find the best split
        for threshold in fvals:

            # The left split will have all data points where the feature value is smaller than threshold
            ind_left = X[:,feature] < threshold
            left_split = X[ind_left,feature]
             # The right split will have all data points where the feature value is larger or equal
            ind_right = X[:,feature] >= threshold
            right_split = X[ind_right,feature]
            
            #compute info-gain for current feature and threshold split
            I = infogain(y,y[ind_left],y[ind_right])
            
            # If this is the best error we have found so far, store the feature as best_feature
            # the threshold as the best threshold and the error as best_error
            if I > best_I:
                best_feature = feature
                best_threshold = threshold
                best_I = I
        
    return best_feature, best_threshold # Return the best feature and threshold


def infogain(yparent,yleft,yright):
    
    Nparent = len(yparent)
    Nleft = len(yleft)
    Nright = len(yright)
    
    #when one of the splits is empty returns I = 0
    if Nleft ==0 or Nright == 0:
        I = 0
    else:
        #compute information gain
        I = entropy(yparent) -( (Nleft/Nparent)*entropy(yleft) + (Nright/Nparent)*entropy(yright) )   

    return I


#entropy for multiple classes
def entropy(y):
    C,unique_counts = np.unique(y,return_counts=True) #the id of classes and number of each
    Pc = unique_counts/len(y)
    H = -(Pc*np.log(Pc)).sum()
    return H    


def create_leaf(target_values,C):

    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True,
            'prediction': None,
            'labels_distribution':None                       }   
    
    # Count the number of data points of each class in the leaf.
    C_in_node,unique_counts = np.unique(target_values,return_counts=True) #the id of classes and number of each
    leaf['prediction'] = C_in_node[np.argmax(unique_counts)]
    
    Classes = np.zeros(C)
    Classes[C_in_node] = unique_counts/len(target_values)
    leaf['labels_distribution'] = Classes
    
    # Return the leaf node        
    return leaf 


def decision_tree_create(X, y, N_features_to_sample, C, min_node_size, Nbins, Verbose, current_depth = 0, max_depth = 10):
    
    #randomly sample a subset of features
    Nfeatures = X.shape[1]
    features = np.random.choice(Nfeatures, N_features_to_sample, replace=False)    
    
    #select only the features sampled for this run
    Xcurrent = X[:,features]
    target_values = y

    if Verbose == True:
        print "--------------------------------------------------------------------"
        print "Subtree, depth = %s (%s data points)." % (current_depth, len(target_values))
        print "Features selected = %s" % features


    # Stopping condition 1
    # (Check if there are mistakes at current node, i.e. if the node is pure.)
    if intermediate_node_num_mistakes(target_values) == 0:  
        if Verbose == True:
            print "No Mistakes at current node - Stopping."     
        # If not mistakes at current node, make current node a leaf node
        return create_leaf(target_values,C)
    
    #Stopping condition 2: min node size reached
    if reached_minimum_node_size(y, min_node_size):
        if Verbose == True:
            print "Minimum node size reached - Stopping"
        return create_leaf(y,C)
    
    # Stopping condition 3: (limit tree depth)
    if current_depth >= max_depth:  
        if Verbose == True:
            print "Reached maximum depth. Stopping."
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values,C)

    # Find the best splitting feature and its threshold
    splitting_feature,splitting_thres = best_splitting_feature(Xcurrent,y,Nbins)
    splitting_feature = features[splitting_feature]
    
    # Split on the best feature that we found. 
    ind_left = X[:,splitting_feature] < splitting_thres
    left_split = X[ind_left,:]
    y_left = y[ind_left]

    ind_right = X[:,splitting_feature] >= splitting_thres
    right_split = X[ind_right,:]
    y_right = y[ind_right]

    if Verbose == True:
        print "Split on feature %s. (%s, %s), Threshold = %s" % (\
        splitting_feature, y_left.shape, y_right.shape, splitting_thres)
    
    # Create a leaf node if the split is "perfect"
    if len(y_left) == len(y) or len(y_right) == len(y):
        if Verbose == True: 
            print 'One split empty: Creating Leaf'          
        return create_leaf(y,C)  
        
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, y_left, N_features_to_sample, C, min_node_size, Nbins, Verbose, current_depth + 1, max_depth)        
    right_tree = decision_tree_create(right_split, y_right, N_features_to_sample, C, min_node_size, Nbins, Verbose, current_depth + 1, max_depth)

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'threshold'        : splitting_thres,
            'left'             : left_tree, 
            'right'            : right_tree,
            'labels_distribution': None 
            
            }

def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

def count_leaves(tree):
    if tree['is_leaf']:
        return 1 
    return count_leaves(tree['left']) + count_leaves(tree['right'])

def classify(tree, x):   
    # if the node is a leaf node.
    if tree['is_leaf']:
        return tree['labels_distribution'] 
    else:
        # split on feature.
        val_split_feature = x[tree['splitting_feature']]
        if val_split_feature < tree['threshold']:
            return classify(tree['left'], x)
        else:
            return classify(tree['right'],x)
        
def evaluate_classification_error_tree(tree, X, y):
    # Apply the classify(tree, x) to each row in your data
    P = map(lambda x: classify(tree,x), X)
    prediction = np.argmax(P,axis=1)
    # Once you've made the predictions, calculate the classification error and return it
    mistakes = sum(prediction != y)
    error = mistakes/len(y)
    return error

### Forest functions

In [116]:
def forest_create(X,y,ntrees,nvarsample=None, min_node_size = 5, Nbins = 10,max_depth=20):
    
    if nvarsample == None:
        nvarsample = (np.round(np.sqrt(X.shape[1]))).astype(int)
        print 'Nfeatures = %s'%nvarsample
    
    #the number of classes is inferred from the data
    C = len(np.unique(y))
    
    nptrain = X.shape[0] #how many datapoints each tree is trained (same size of X)
    RF = []
    #for loop creating and training each tree 
    #bootstrap X to train each tree
    for t in range(ntrees):
        print 'current trained tree = %s'%t
        #create bootstrap training dataset for tree t
        indbootstrap = np.random.choice(X.shape[0],nptrain)
        Xtree = X[indbootstrap,:]
        ytree = y[indbootstrap] 
        
        #train the tree
        tree1 = decision_tree_create(Xtree,ytree,nvarsample,C,min_node_size,Nbins,Verbose=False,max_depth = max_depth)
        RF.append(tree1)
    
    print 'Forest Trained!'
    return RF
    

#outputs the posterior prob of each tree and the corresponding class
def forest_posterior(RF,x):

    T = len(RF)  #the number of trees 

    #infer the number of classes
    P0 = classify(RF[0],x)
    C = len(P0)
    
    Pt = np.zeros((T,C)) #matrix of posteriors from each tree (T x Nclasses)
    Pt[0,:] = P0
    for t in range(len(RF))[1:]:
        Pt[t,:] = classify(RF[t],x) 
    return Pt
 
    
#classify input based on majority voting of each tree prediction
def forest_classify_majority(RF,x):
        Pt = forest_posterior(RF,x)
        Yt = np.argmax(Pt,axis=1)         
        C,unique_counts = np.unique(Yt,return_counts=True) #the id of classes and number of each
        return C[np.argmax(unique_counts)]   
    
#classify input by averaging posteriors 
def forest_classify_ensemble(RF,x):
    Pt = forest_posterior(RF,x)
    Pforest = Pt.mean(axis=0)
    ypred = np.argmax(Pt.mean(axis=0))
    return ypred

def evaluate_classification_error(RF, X, y, method = None):  
    # Apply the forest_classify(RF, x) to each row in your data
    if method == None:
        ypred = map(lambda x: forest_classify_ensemble(RF,x), X)
        # Once you've made the predictions, calculate the classification error and return it
        mistakes = sum(ypred != y)
        error = mistakes/len(y)
    return error

### SER Algorithm functions

In [58]:
#Refine the trained forest on target data using the SER algorithm
def forest_SER(RF,XT,yT,C,max_depth=2):

    nptrain = len(yT) #how many datapoints each tree is trained (same size of yT)
    ntrees = len(RF)
    RFnew = []

    for t in range(ntrees):
        print 'expanding/reducing tree = %s'%t
        #Bootstrap XT1 and XT2
        indbootstrap1 = np.random.choice(XT.shape[0],nptrain)
        indbootstrap2 = np.random.choice(XT.shape[0],nptrain)
        XT1 = XT[indbootstrap1,:]
        XT2 = XT[indbootstrap2,:]
        yT1 = yT[indbootstrap1]
        yT2 = yT[indbootstrap2]

        treeNew = expansion_reduction(RF[t],XT1,yT1,XT2,yT2,max_depth,C)
        RFnew.append(treeNew)
        
    print 'Forest refined on target data!'
    return RFnew


# compute the path to the leaf followed by data point x  
def datapath(tree, x, branch = 1):   
    # if the node is a leaf node.
    if tree['is_leaf']:
        return branch 
    else:
        # split on feature.
        split_feature = tree['splitting_feature']
        split_threshold = tree['threshold']

        if x[split_feature] < split_threshold:
            return datapath(tree['left'], x, 2*branch)
        else:
            return datapath(tree['right'],x, 2*branch+1)

        
def expansion_reduction(tree,XT1,yT1,XT2,yT2,max_depth=2,C=-1,min_node_size = 5):

    #Tree = tree #a copy of the tree
    
    #finding the leaf where each target datapoint ends up
    leavesData1 = map(lambda x: datapath(tree,x), XT1)
    leavesData2 = map(lambda x: datapath(tree,x), XT2)
            
    Uleaves1 = np.unique(leavesData1)  #the path to each leaf followed by data1
    Uleaves2 = np.unique(leavesData2)  #the path to each leaf followed by data2
    Uleaves = list(set(Uleaves1) & set(Uleaves2)) #leaves reached by both data1 and data2
            
    #expanding each leaf on the 1st bootstrap replica of target data
    for i in Uleaves:
        ind_data1 = leavesData1==i #indices of datapoints for each leaf
        #set the current depth of subtree to 1 to prevent inferring the number of classes from the code
        Exp_tree = decision_tree_create(XT1[ind_data1,:],yT1[ind_data1],XT1.shape[1], C, min_node_size, Nbins = 10, Verbose = False, current_depth=1,max_depth=max_depth)

        #Is this a good expansion?: computes classification error at each leaf for Data T2
        ind_data2 = leavesData2==i
        Err_leavesT2 = intermediate_node_num_mistakes(yT2[ind_data2])/len(yT2[ind_data2])

        #error at the current subtree on Data T2
        Err_subtreeT2 = evaluate_classification_error_tree(Exp_tree, XT2[ind_data2], yT2[ind_data2])
        
        #comparing the error of the subtree with that at the leaf node of the original tree
        if Err_subtreeT2 < Err_leavesT2:
            tree = mergetrees(tree,i,Exp_tree)
            print 'merging successful!'
        else:
            print 'no merging: discard subtree'
    
    return tree


def mergetrees(tree1,leafnr,tree2):
    leafnrbin = bin(leafnr)[3:]  #path is from the 4th element of the binary on: 0 = go left, 1 = go right
    path = ''
    for i in range(len(leafnrbin)):
        if leafnrbin[i] == '0':
            path=path+str("['left']")
        else:
            path=path+str("['right']") 
    # print(path)
    exec ('tree1'+path+"['prediction']"+'=None')
    exec ('tree1'+path+"['is_leaf']"+'=False')
    exec ('tree1'+path+"['left']"+"=tree2['left']")
    exec ('tree1'+path+"['right']"+"=tree2['right']")
    exec ('tree1'+path+"['splitting_feature']"+"=tree2['splitting_feature']")
    exec ('tree1'+path+"['threshold']"+"=tree2['threshold']")
    exec ('del(tree1'+path+"['labels_distribution'])")
    
#    print ('tree1'+path+"['prediction']"+'=None')
#    print ('tree1'+path+"['is_leaf']"+'=False')
#    print ('tree1'+path+"['left']"+"=tree2['left']")
#    print ('tree1'+path+"['right']"+"=tree2['right']")
#    print ('tree1'+path+"['splitting_feature']"+"=tree2['splitting_feature']")
    
    #print('tree1'+path+'=tree2')
    return tree1

### Generate a fake dataset 

In [18]:
#artificial training dataset
Xtrain = np.random.rand(1000,10)
y = np.ones(1000).astype(int)
y[(Xtrain[:,0] < 0.25)] = 0
y[(Xtrain[:,1] > 0.55)] = 2
np.unique(y)

array([0, 1, 2])

### A linearly separable dataset

In [9]:
Xtrain = np.random.rand(1000,10)
y = np.ones(1000).astype(int)
y[(Xtrain[:,0]+Xtrain[:,1]+Xtrain[:,2] < 1)] = 0
y[(Xtrain[:,0]+3*Xtrain[:,2] >= 1.5)] = 2
y[(1.5*Xtrain[:,0]-2*Xtrain[:,1]+Xtrain[:,2] < 1)] = 3

### Another fake dataset with more features

In [506]:
X,y = make_classification(n_samples=2000,n_features=10,n_informative=5,n_classes=2)
print X.shape
print np.unique(y) #original labels are 0 and 1
ind = y==0
y[ind] = -1
print np.unique(y)

(2000, 10)
[0 1]
[-1  1]


## Train-test split + Split test data into target1 (for expansion) and target2 (for reduction)

### Now we can train on our version of random forest to compare the accuracy

In [11]:
mistakes = sum(ypred != ytest)
error = mistakes/len(ytest)
print error

0.109490909091
